<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Bi-directional-problem" data-toc-modified-id="Bi-directional-problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bi-directional problem</a></span><ul class="toc-item"><li><span><a href="#Run" data-toc-modified-id="Run-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run</a></span><ul class="toc-item"><li><span><a href="#Init-Graph" data-toc-modified-id="Init-Graph-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Init Graph</a></span></li><li><span><a href="#Run-ICU" data-toc-modified-id="Run-ICU-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Run ICU</a></span></li><li><span><a href="#Run-Outer-Cost-Update" data-toc-modified-id="Run-Outer-Cost-Update-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Run Outer Cost Update</a></span></li></ul></li></ul></li></ul></div>

# Description

This file aims at implementing both versions of the algorithm (with rebalancers update inside and outside of the loop) and at comparing their performance and convergence to a solution. 

Comparison is done on a simple graph. The dummy problem we talked about. 

# Load

In [8]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from routines_icu import *
from helpers_icu import *
import cvxpy as cp
from FW_icu import *

# Bi-directional problem

In [9]:
#parameters
alpha=0.15
beta=4
POTENTIAL_SHIFT=90
INVERSE_DEMAND_SHIFT=80

ZC_EDGE=1

In [10]:
L_dum=10
#to adapt
#to be read from file if necessary
nodes_list=['1','2','2_p','R','1_p']
edge_list=[('1','2'),('2','1'),('2','2_p'),('1','R'),('1','2_p'),('1','1_p'),('2','1_p'),('2','R')]
k_list=[10,10,3,10,ZC_EDGE,2,ZC_EDGE,10] 
l_list=[10,10,1,L_dum,0,1,0,L_dum]
t_list=[10,10,10,10,ZC_EDGE,10,ZC_EDGE,10]
phi_list=[phi(l_list[i],t_list[i]) for i in range(len(l_list))]
is_negative=[0,0,1,0,0,1,0,0]

nodes_pots=[('2_p',POTENTIAL_SHIFT),('1_p',POTENTIAL_SHIFT)]

In [11]:
OD=dict()
N=10
OD[('1','2_p')]=N
OD[('2','R')]=N
OD[('2','1_p')]=N
OD[('1','R')]=N

print(OD)

{('1', '2_p'): 10, ('2', 'R'): 10, ('2', '1_p'): 10, ('1', 'R'): 10}


## Run

### Init Graph

In [ ]:
OD=dict()
N=10
OD['1','2_p']=N
OD['2','1_p']=N

In [ ]:
G=nx.DiGraph()
G.add_nodes_from(nodes_list)
G.add_edges_from(edge_list)

In [ ]:
G=initEdgeAttr(G,edge_list,k_list,phi_list,is_negative)
G=initNodeAttr(G,nodes_pots)
G=update_costs(G,INVERSE_DEMAND_SHIFT)

#Create a feasible solution for the passengers
G=init_flows(G,OD) 

# for e in G.edges():
#     print(e," --- ",G.edges[e])

G=update_costs(G,INVERSE_DEMAND_SHIFT)

dummy_nodes=dict()
dummy_nodes['2_p']='2'
dummy_nodes['1_p']='1'

### Run ICU

In [ ]:
G_k,y_list,opt_res,OD_list=modified_FW(G,OD,edge_list,dummy_nodes,maxIter=400,
                         step='fixed',rebalancer_smoothing=True, ri_smoothing=False,evolving_bounds=True)

### Run Outer Cost Update